In [ ]:
from api_request_funtions import ApiGet
from bq_transfers import BqDataTransfers 

In [ ]:
thrivecart_get = ApiGet(http='thrivecart.com', api_key='TZ5TJYBR-FDB85IBI-0RFTB00N-VQ7ZFY2S')
thrivecart_save = BqDataTransfers(gcp_project_id= 'arboreal-cat-451816-n0', bq_data_set = 'thrive_cart')
bq_client = thrivecart_save.get_bq_client("C:/Users/shami/iCloudDrive/Personal Projects/vexis/vexis_bq_writter.json")

In [21]:
# products
x = thrivecart_get.fetch_data(end_point = f'/api/external/products/{id}')
df = thrivecart_get.process_reponse_df(x)
df = thrivecart_get.fix_data_types(df)


C:\Users\shami\PersonalProjects\upwork\vexis_authLive\api_request_funtions.py:57: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Users\shami\PersonalProjects\upwork\vexis_authLive\api_request_funtions.py:57: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Users\shami\PersonalProjects\upwork\vexis_authLive\api_request_funtions.py:57: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Users\shami\PersonalProjects\upwork\vexis_authLive\api_request_funtions.py:57: FutureWarning: errors='ignore' is

In [20]:
x

{'product_id': '285',
 'name': 'Quantum Creation 8-Week Advanced Manifesting Experience',
 'slug': 'quantum-creation-1111',
 'label': 'QC 8-Week 1/2025 1,111',
 'status': '3',
 'statusString': 'disabled',
 'type': '1',
 'typeString': 'digital',
 'payment_currency': 'USD',
 'url': 'https://mandymorris.thrivecart.com/quantum-creation-1111/',
 'embed_type': 'standard'}

In [22]:
df

[{'product_id': '409',
  'name': 'EME Integration Level 2 Practitioner Certification Program',
  'label': 'EME 2 7/2025',
  'url': 'https://mandymorris.thrivecart.com/eme-2-72025/',
  'embed_type': 'standard',
  'status': '2',
  'statusString': 'live',
  'type': '1',
  'typeString': 'digital'},
 {'product_id': '407',
  'name': 'Geo Love Healing Level 2 Practitioner Certification Program',
  'label': 'Geo 2 7/2025 4555 PIF ST',
  'url': 'https://mandymorris.thrivecart.com/geo-2-72025-pif-st/',
  'embed_type': 'standard',
  'status': '2',
  'statusString': 'live',
  'type': '1',
  'typeString': 'digital'},
 {'product_id': '406',
  'name': 'Geo Love Healing Level 2 Practitioner Certification Program ($4,555) Split',
  'label': 'Geo 2 (7/2025) 4555 Split',
  'url': 'https://mandymorris.thrivecart.com/glh-2-72025-split/',
  'embed_type': 'standard',
  'status': '2',
  'statusString': 'live',
  'type': '1',
  'typeString': 'digital'},
 {'product_id': '405',
  'name': 'Geo Love Healing Level 

In [9]:
thrivecart_save.start_transfer(bq_client=bq_client ,df = df, destination_table = 'upsells')